<a href="https://colab.research.google.com/github/matheuspercario/TT007-AnaliseDadosPython/blob/master/exercicios/Aula5-POO/liarsdice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício - Liars Dice


Criado por: Matheus Bruder

## Instruções de jogo

No Liers’ dice, cada jogador recebe 5 dados. Em cada rodada os jogadores lançam os seus dados, mantendo suas faces escondidas dos demais jogadores.
Cada jogador então escolhe uma face (1 a 6) e da um palpite de quantas faces iguais a esta há na mesa.
O próximo a jogar tem três alternativas:
  * _Escolhe uma face e da um palpite, que deve ser necessariamente maior que o palpite anterior e passa para o próximo jogador._
  * _Chama o jogador anterior de mentiroso, neste caso abrem-se os dados e verifica-se o palpite corrente é verdadeiro. Se for o jogador atual perde um dado, senão o jogador anterior perde um dado._
  * _Concorda com o palpite do jogador anterior. Neste caso, abrem-se os dados e se o palpite estiver correto, o jogador anterior perde dois dados._

Ganha o último jogador com dados na mesa.

### Informações extras
* O jogo deve ser para um usuário contra até 4 jogadores controlados pelo computador;
* Use Objetos;
* Considere usar probabilidades para que um jogador do PC, de um palpite, chame de mentiroso ou concorde com o jogador anterior;
* Use diferentes probabilidade para diferentes jogadores para simular diferentes comportamentos;
* É possível utilizar-se de alguma heurística para melhorar os jogadores do PC?
* Quem fica mais tempo na mesa, um jogador com heurística ou um jogador aleatório?

---

In [ ]:
# Iniciar códigos aqui

In [ ]:
class Player:

    def __init__(self, name, num_die=5, dices):
        self.name = name
        self.num_die = num_die
        self.dices = dices
    